In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


# Improvements from v.1
    - Optimize Regression Tree
    
Regression tree when left idle will automatically set max_depth to 'none'. Thi allows the trees to run on until they can't evaluate any more leaves. I get 0.03 increase in model score by allowing the model to run for 30 minutes extra. Hardly seems worth it.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error

In [3]:
# Read datasets
train_data = pd.read_csv('/kaggle/input/30-days-of-ml/train.csv')
test_data = pd.read_csv('/kaggle/input/30-days-of-ml/test.csv')

In [4]:
# Check data
train_data.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Check shapes
train_data.shape, test_data.shape

((300000, 26), (200000, 25))

In [6]:
# Check columns
train_data.columns, test_data.columns

(Index(['id', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
        'cat8', 'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
        'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12',
        'cont13', 'target'],
       dtype='object'),
 Index(['id', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
        'cat8', 'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
        'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12',
        'cont13'],
       dtype='object'))

In [7]:
# Copy datasets for altering
train_full = train_data.copy()
test_full = test_data.copy()

In [8]:
# Checking Features to drop - Removing high cardinality
cardinality = {}
for col in train_full.columns:
    # Only concenred with object cols
    if train_full[col].dtypes == 'object':
        cardinality[col] = len(train_full[col].unique())

In [9]:
# Cat9 needs to be dropped
train_full.drop('cat9', axis = 1, inplace = True)
test_full.drop('cat9', axis = 1, inplace = True)

In [10]:
# Seperate features and targets
y = train_full['target']
X = train_full.drop('target', axis = 1)

In [11]:
# Create train set and validation set
X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size = 0.6, random_state = 0)

In [12]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((180000, 24), (120000, 24), (180000,), (120000,))

In [13]:
# Extract object cols
category_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
category_cols

['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8']

In [14]:
# Encoding Category Cols
enc = OrdinalEncoder()

cat_X_train = X_train.copy()
cat_X_valid = X_valid.copy()

cat_X_train[category_cols] = enc.fit_transform(X_train[category_cols])
cat_X_valid[category_cols] = enc.transform(X_valid[category_cols])
test_full[category_cols] = enc.transform(test_full[category_cols])

In [15]:
## Runs for approx 2.5 hours. 

estimators = [600, 700]
depth  = [6, 7, 8]

def models_reg(estimators, depth):
    results = {}

    for i in range(len(estimators)):
        for j in range(len(depth)):
            start = time.time()
            
            model = RandomForestRegressor(n_estimators = estimators[i], max_depth = depth[j], random_state = 0)
            model.fit(cat_X_train, y_train)
            y_predict = model.predict(cat_X_valid)
            score = mean_squared_error(y_valid, y_predict)
    
            results['Estimators: ' + str(estimators[i]) + ' Depth: ' + str(depth[j])] = score
        
            print('Estimators: ' + str(estimators[i]) + ' Depth: ' + str(depth[j]) + ' Score: ' + str(score))
            end = time.time()
            print(str(round((end - start)/60, 2)) + ' minutes')
            
    return results

In [16]:
models_reg(estimators, depth)

Estimators: 600 Depth: 6 Score: 0.5437429843125167
19.73 minutes
Estimators: 600 Depth: 7 Score: 0.5424125629586826
22.06 minutes
Estimators: 600 Depth: 8 Score: 0.5412569380842072
24.3 minutes
Estimators: 700 Depth: 6 Score: 0.543724475908459
22.27 minutes
Estimators: 700 Depth: 7 Score: 0.5423882595559154
25.03 minutes
Estimators: 700 Depth: 8 Score: 0.5412358427075951
28.32 minutes


{'Estimators: 600 Depth: 6': 0.5437429843125167,
 'Estimators: 600 Depth: 7': 0.5424125629586826,
 'Estimators: 600 Depth: 8': 0.5412569380842072,
 'Estimators: 700 Depth: 6': 0.543724475908459,
 'Estimators: 700 Depth: 7': 0.5423882595559154,
 'Estimators: 700 Depth: 8': 0.5412358427075951}

In [17]:
## Runs for 1 hour and 7 minutes. 

start = time.time()

model = RandomForestRegressor(n_estimators = 700)
model.fit(cat_X_train, y_train)
y_predict = model.predict(cat_X_valid)
score = mean_squared_error(y_valid, y_predict)
print(score)

end = time.time()
print(str(round((end - start)/60, 2)) + ' minutes')

0.5368497549036454
64.47 minutes


In [18]:
test_predictions = model.predict(test_full)

In [19]:
output = pd.DataFrame({'Id': test_full['id'], 
                      'Target': test_predictions})

In [20]:
output.to_csv('my_submission.csv', index = False)